In [28]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from getpass import getuser


In [29]:
# Get the current user's name
user = getuser()
# Paths to the datasets
fpl_data_path = rf'C:\Users\{user}\Documents\GitHub\dream-team-fpl-prediction\data\fpl_data.xlsx'
players_agg_path = rf'C:\Users\{user}\Documents\GitHub\dream-team-fpl-prediction\data\players_agg.xlsx'
team_stats_path = rf'C:\Users\{user}\Documents\GitHub\dream-team-fpl-prediction\data\teams_stats.xlsx'
def_player_stats_path = rf'C:\Users\{user}\Documents\GitHub\dream-team-fpl-prediction\data\def\player_defense_stats.xlsx'
team_def_stats_path = rf'C:\Users\{user}\Documents\GitHub\dream-team-fpl-prediction\data\def\team_defense_stats.xlsx'


In [30]:
# Read the datasets
fpl_data = pd.read_excel(fpl_data_path)
players_agg = pd.read_excel(players_agg_path)
team_stats = pd.read_excel(team_stats_path)
def_player_stats = pd.read_excel(def_player_stats_path)
team_def_stats = pd.read_excel(team_def_stats_path)

In [31]:
# Replace "PetroviÄ" with "Đorđe Petrović" wherever it appears as part of a player's name
fpl_data['player'] = np.where(fpl_data['player'].str.contains('Chris LÃ¶we', na=False), 
                              'Chris Löwe', 
                              fpl_data['player'])

players_agg['player'] = np.where(players_agg['player'].str.contains('Chris LÃ¶we', na=False), 
                              'Chris Löwe', 
                              players_agg['player'])


In [32]:
# merge players and team stats datasets
pl_def_df = pd.merge(def_player_stats, team_def_stats, left_on=['season_player', 'squad_player'], right_on=['season', 'squad'])
pl_def_df

,season_player,player,nation_player,squad_player,age_player,born_player,90s_player,tkl_player,tklw_player,tkl_def_player,...,challenges_att,tkl_pct,challenges_lost,blocks,shots_blocked,passes_blocked,interceptions,tackles_interceptions,clearances,errors
0,2016-2017,Patrick van Aanholt,NED,Crystal Palace,25.0,1990.0,7.9,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,575,NaN,NaN,NaN
1,2016-2017,Christian Benteke,BEL,Crystal Palace,25.0,1990.0,34.8,NaN,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,575,NaN,NaN,NaN
2,2016-2017,Jonathan Benteke,BEL,Crystal Palace,21.0,1995.0,0.1,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,575,NaN,NaN,NaN
3,2016-2017,Yannick Bolasie,COD,Crystal Palace,27.0,1989.0,0.3,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,575,NaN,NaN,NaN
4,2016-2017,Yohan Cabaye,FRA,Crystal Palace,30.0,1986.0,23.6,NaN,39.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,575,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898,2024-2025,Takehiro Tomiyasu,JPN,Arsenal,25.0,1998.0,0.1,0.0,0.0,0.0,...,515.0,50.5,255.0,328.0,106.0,222.0,225,823.0,680.0,33.0
4899,2024-2025,Leandro Trossard,BEL,Arsenal,29.0,1994.0,28.3,31.0,20.0,6.0,...,515.0,50.5,255.0,328.0,106.0,222.0,225,823.0,680.0,33.0
4900,2024-2025,Ben White,ENG,Arsenal,26.0,1997.0,13.3,20.0,14.0,10.0,...,515.0,50.5,255.0,328.0,106.0,222.0,225,823.0,680.0,33.0
4901,2024-2025,Oleksandr Zinchenko,UKR,Arsenal,27.0,1996.0,5.9,6.0,1.0,1.0,...,515.0,50.5,255.0,328.0,106.0,222.0,225,823.0,680.0,33.0


In [33]:
# Convert all relevant columns to lowercase
fpl_data.columns = fpl_data.columns.str.lower()
players_agg.columns = players_agg.columns.str.lower()
team_stats.columns = team_stats.columns.str.lower()

# List of seasons to analyze (2016-2017 to 2024-2025)
seasons = [f'{year}-{year+1}' for year in range(2016, 2025)]

# Initialize lists to store the final merged data and matched goalkeepers for all seasons
final_merged_data = []
all_matched_defs = []

In [34]:
# Function to get the best fuzzy match for a player name, with a threshold of 73
def get_best_match(player_name, player_list, threshold=73):
    result = process.extractOne(player_name, player_list, scorer=fuzz.token_sort_ratio)
    if result and result[1] >= threshold:  # Ensure score meets the threshold
        return result[0], result[1]  # Return both the matched player name and the score
    else:
        return None, None  # Explicitly return None for both values

In [35]:
# Iterate through each season
for season in seasons:
    # Filter for the current season
    fpl_season_data = fpl_data[fpl_data['season'] == season]
    players_agg_season_data = players_agg[(players_agg['season'] == season) & 
                                          (players_agg['pos'] == 'DF') &  # Ensure correct case for 'DF' - defenders
                                          (players_agg['mp_player'].between(10, 38))]

    # Filter out players from fpl_data where all columns except 'player', 'season', and 'selected_by_percent' are zero
    fpl_season_data = fpl_season_data[(fpl_season_data.drop(columns=['player', 'season', 'selected_by_percent']) != 0).any(axis=1)]

    # Keep only those observations where 'total_points' is in the 60% percentile
    percentile_60 = fpl_season_data['total_points'].quantile(0.6)
    players_below_60_percentile = fpl_season_data[fpl_season_data['total_points'] < percentile_60]['player']
    fpl_season_data = fpl_season_data[fpl_season_data['total_points'] >= percentile_60]

    # Now, we will remove the observations from `players_agg_season_data` that have these player names.
    players_agg_season_data = players_agg_season_data[~players_agg_season_data['player'].isin(players_below_60_percentile)]

    # List to store the results
    matched_players = []

    # Iterate through each player in the players_agg_season_data dataset (to match players_agg to fpl)
    for player in players_agg_season_data['player']:
        best_match, score = get_best_match(player, fpl_season_data['player'].tolist())
        if best_match and score:  # Ensure we have a valid match and score
            matched_players.append((player, best_match, score))

    # Convert the matched players list to a DataFrame
    matched_players_df = pd.DataFrame(matched_players, columns=['player_agg', 'player_fpl', 'score'])

    # Remove any duplicates in the matched_players_df based on Player_Agg
    matched_players_df = matched_players_df.drop_duplicates(subset=['player_agg'])

    # Store the matched goalkeepers for the season
    matched_players_df['season'] = season
    all_matched_defs.append(matched_players_df)

    # Rename the 'player' column in the players_agg_season_data to avoid conflicts
    players_agg_season_data.rename(columns={'player': 'player_agg'}, inplace=True)

    # Merge the datasets with the renamed column
    merged_fpl = fpl_season_data.merge(matched_players_df, left_on='player', right_on='player_fpl')
    merged_data = merged_fpl.merge(players_agg_season_data, left_on='player_agg', right_on='player_agg', suffixes=('_fpl', '_agg'))

    # Remove the player_fpl column and rename player_agg to player
    merged_data.drop(columns=['player_fpl'], inplace=True)
    merged_data.rename(columns={ 'score': 'matching_name_score'}, inplace=True)

    # Remove season_x and season_y columns and keep one season column
    if 'season_x' in merged_data.columns and 'season_y' in merged_data.columns:
        merged_data['season'] = merged_data['season_x'].combine_first(merged_data['season_y'])
        merged_data.drop(columns=['season_x', 'season_y'], inplace=True)

    # Append the season's merged data to the final list
    final_merged_data.append(merged_data)

    # Concatenate all the season data into one final DataFrame
final_merged_data_df = pd.concat(final_merged_data, ignore_index=True)

In [36]:
# Remove specified columns from the final DataFrame
columns_to_remove = [ 'pos_fpl', 'first_name', 'second_name', 'player_agg', 'matching_name_score', 'pos_agg', 'minutes',
                     'gls_player', 'ast_player', 'g+a_player', 'yellow_cards', 'red_cards', 'season_player'
]
final_merged_data_df.drop(columns=columns_to_remove, inplace=True, errors='ignore')


In [37]:
# Merge defenders dataset (fpl +standard stats) with team_stats
final_merged_data_df = final_merged_data_df.merge(team_stats, on=['season', 'squad'], how='left')

In [38]:
final_merged_data_df = final_merged_data_df.rename(columns={
    'xg_90minutes_team': 'xg_90min_team',
    'xag_90minutes_team': 'xag_90min_team',
    'npxg_90minutes_team': 'npxg_90min_team',
    'npxg+xag_90minutes_team': 'npxg+xag_90min_team'
})


In [39]:
final_merged_data_df.columns

Index(['player', 'goals_scored', 'assists', 'total_points', 'goals_conceded',
       'creativity', 'influence', 'threat', 'bonus', 'bps', 'ict_index',
       'clean_sheets', 'selected_by_percent', 'cost', 'season', 'nation',
       'squad', 'age_player', 'born', 'mp_player', 'starts_player',
       'min_player', '90s_player', 'g-pk_player', 'pk_player', 'pkatt_player',
       'crdy_player', 'crdr_player', 'gls_90min_player', 'ast_90min_player',
       'g+a_90min_player', 'g-pk_90min_player', 'g+a-pk_player', 'xg_player',
       'npxg_player', 'xag_player', 'npxg+xag_player', 'prgc_player',
       'prgp_player', 'prgr_player', 'xg_90min_player', 'xag_90min_player',
       'xg+xag_player', 'npxg_90min_player', 'npxg+xag_90min_player',
       'observationcount_player', 'num_players_team', 'avg_age_team',
       'poss_team', 'gls_team', 'ast_team', 'g+a_team', 'g-pk_team', 'pk_team',
       'pkatt_team', 'crdy_team', 'crdr_team', 'gls_90minutes_team',
       'ast_90minutes_team', 'g+a_90mi

In [40]:
# Drop duplicate columns
def_player_stats = def_player_stats.drop(columns=[ 'nation_player', 'squad_player', 'age_player', 'born_player', '90s_player'])


In [41]:
def_player_stats.columns

Index(['season_player', 'player', 'tkl_player', 'tklw_player',
       'tkl_def_player', 'tkl_mid_player', 'tkl_att_player',
       'challenges_won_player', 'challenges_att_player', 'tkl_pct_player',
       'challenges_lost_player', 'blocks_player', 'shots_blocked_player',
       'passes_blocked_player', 'interceptions_player',
       'tackles_interceptions_player', 'clearances_player', 'errors_player',
       'pos_player_1', 'pos_player_2'],
      dtype='object')

In [42]:
# Merge defenders dataset with defensive actions players
final_merged_data_df = final_merged_data_df.merge(def_player_stats, 
                                                  left_on=['season', 'player'], 
                                                  right_on=['season_player', 'player'], 
                                                  how='left')


In [43]:
# Drop duplicate columns from team_def_stats
final_merged_data_df = final_merged_data_df.drop(columns=[ 'season_player'])

In [44]:
# Drop duplicate columns
team_def_stats = team_def_stats.drop(columns=[ 'np', '90s'])


In [45]:
# add _team suffix to all columns in team_def_stats
team_def_stats = team_def_stats.add_suffix('_team')


In [46]:
team_def_stats.columns

Index(['season_team', 'squad_team', 'tkl_team', 'tklw_team', 'tkl_def_team',
       'tkl_mid_team', 'tkl_att_team', 'challenges_won_team',
       'challenges_att_team', 'tkl_pct_team', 'challenges_lost_team',
       'blocks_team', 'shots_blocked_team', 'passes_blocked_team',
       'interceptions_team', 'tackles_interceptions_team', 'clearances_team',
       'errors_team'],
      dtype='object')

In [47]:
final_merged_data_df.columns

Index(['player', 'goals_scored', 'assists', 'total_points', 'goals_conceded',
       'creativity', 'influence', 'threat', 'bonus', 'bps', 'ict_index',
       'clean_sheets', 'selected_by_percent', 'cost', 'season', 'nation',
       'squad', 'age_player', 'born', 'mp_player', 'starts_player',
       'min_player', '90s_player', 'g-pk_player', 'pk_player', 'pkatt_player',
       'crdy_player', 'crdr_player', 'gls_90min_player', 'ast_90min_player',
       'g+a_90min_player', 'g-pk_90min_player', 'g+a-pk_player', 'xg_player',
       'npxg_player', 'xag_player', 'npxg+xag_player', 'prgc_player',
       'prgp_player', 'prgr_player', 'xg_90min_player', 'xag_90min_player',
       'xg+xag_player', 'npxg_90min_player', 'npxg+xag_90min_player',
       'observationcount_player', 'num_players_team', 'avg_age_team',
       'poss_team', 'gls_team', 'ast_team', 'g+a_team', 'g-pk_team', 'pk_team',
       'pkatt_team', 'crdy_team', 'crdr_team', 'gls_90minutes_team',
       'ast_90minutes_team', 'g+a_90mi

In [48]:
# Merge defenders dataset with defensive actions players
final_merged_data_df = final_merged_data_df.merge(team_def_stats, 
                                                  left_on=['season', 'squad'], 
                                                  right_on=['season_team', 'squad_team'], 
                                                  how='left')


In [49]:
# Drop duplicate columns from team_def_stats
final_merged_data_df = final_merged_data_df.drop(columns=[ 'season_team', 'squad_team'])


In [50]:
# Desired order of columns
desired_order = [
    # fpl stats
  'player', 'season', 'goals_scored', 'assists', 'total_points','goals_conceded', 'creativity', 'influence', 
    'threat', 'bonus', 'bps', 'ict_index', 'clean_sheets','selected_by_percent', 'cost',
    # player info and performance 
  'nation', 'squad', 'age_player', 'born', 'mp_player', 'starts_player', 'min_player',
    '90s_player', 'g-pk_player', 'pk_player', 'pkatt_player','crdy_player', 'crdr_player',
    # expected stats player
    'xg_player', 'npxg_player', 'xag_player', 'npxg+xag_player',
    # expected 90 minutes player
    'xg_90min_player', 'npxg_90min_player', 'xag_90min_player', 'npxg+xag_90min_player',
    # progression player 
    'prgc_player', 'prgp_player', 'prgr_player',
    # count observation player 
    'observationcount_player',
    # team stats
    'num_players_team', 'avg_age_team', 'poss_team', 
    'gls_team', 'ast_team', 'g+a_team', 'g-pk_team', 'pk_team', 'pkatt_team', 'crdy_team', 'crdr_team',
    # 90 minutes team stats
    'gls_90minutes_team', 'ast_90minutes_team', 'g+a_90minutes_team', 'g-pk_90minutes_team', 'g+a-pk_team', 
     # expected
    'xg_team', 'npxg_team', 'xag_team', 'npxg+xag_team', 'prgc_team', 'prgp_team', 
    # expected 90 minutes team
    'xg_90min_team', 'xag_90min_team', 'xg+xag_team', 'npxg_90min_team', 'npxg+xag_90min_team', 
    # tackles
    'tkl_player', 'tklw_player','tkl_def_player', 'tkl_mid_player', 'tkl_att_player',
    # challenges 
    'challenges_won_player', 'challenges_att_player', 'tkl_pct_player', 'challenges_lost_player', 'blocks_player',
    # block 
     'challenges_won_player', 'challenges_att_player', 'tkl_pct_player', 'challenges_lost_player', 'blocks_player',
     'shots_blocked_player','passes_blocked_player', 'interceptions_player','tackles_interceptions_player', 
     'clearances_player', 'errors_player',
    # positions 
     'pos_player_1', 'pos_player_2',
     # tackles team
    'tkl_team', 'tklw_team','tkl_def_team', 'tkl_mid_team', 'tkl_att_team',
    # challenges team
    'challenges_won_team', 'challenges_att_team', 'tkl_pct_team', 'challenges_lost_team', 'blocks_team',
    # blocks team
    'blocks_team', 'shots_blocked_team', 'passes_blocked_team', 'interceptions_team', 
    'tackles_interceptions_team', 'clearances_team','errors_team'
]

# Reorder the columns
final_merged_data_df = final_merged_data_df[desired_order]

In [51]:
# Save the final merged dataset to an Excel file
output_path = rf'C:\Users\{user}\Documents\GitHub\dream-team-fpl-prediction\data\final_ds\ds_def.xlsx'
final_merged_data_df.to_excel(output_path, index=False)

In [57]:
df = final_merged_data_df.copy()  # Create a copy of the final DataFrame for further use

In [58]:
# Convert the necessary columns to numeric (in case they are not)
cols = ['clearances_team', 'interceptions_team', 'shots_blocked_team', 'passes_blocked_team']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

# Create CBIT metric as the sum of all components
df['cbit_metric'] = df['clearances_player'] + df['interceptions_player'] + \
                    df['shots_blocked_player'] + df['tkl_player']
# CBIT = clearances + interceptions + shots_blocked + passes_blocked

# Show the top CBIT values
df[['player', 'season', 'cbit_metric']].sort_values(by='cbit_metric', ascending=False).head(10)


,player,season,cbit_metric
123,Shane Duffy,2017-2018,472.0
144,Christopher Schindler,2017-2018,463.0
122,Lewis Dunk,2017-2018,426.0
216,James Tarkowski,2018-2019,409.0
228,Aaron Wan-Bissaka,2018-2019,390.0
215,Ben Mee,2018-2019,388.0
602,James Tarkowski,2022-2023,381.0
178,Kurt Zouma,2017-2018,380.0
299,James Tarkowski,2019-2020,380.0
219,Sol Bamba,2018-2019,369.0


In [59]:
# Filter the dataframe for seasons 2024-2025 or 2023-2024
filtered_df = df[df['season'].isin(['2024-2025', '2023-2024'])]

# Display the top 10 players by CBIT metric for the filtered seasons
top_cbit_recent_seasons = filtered_df[['player', 'season', 'cbit_metric']].sort_values(by='cbit_metric', ascending=False).head(10)
top_cbit_recent_seasons


,player,season,cbit_metric
715,James Tarkowski,2023-2024,364.0
822,James Tarkowski,2024-2025,360.0
832,Dara O'Shea,2024-2025,357.0
797,Nathan Collins,2024-2025,357.0
818,Maxence Lacroix,2024-2025,351.0
868,Max Kilman,2024-2025,329.0
703,Joachim Andersen,2023-2024,322.0
857,Jan Bednarek,2024-2025,318.0
710,Jarrad Branthwaite,2023-2024,315.0
796,Dean Huijsen,2024-2025,314.0


In [56]:
df.columns.tolist()  # Display all columns in the DataFrame

['player',
 'season',
 'goals_scored',
 'assists',
 'total_points',
 'goals_conceded',
 'creativity',
 'influence',
 'threat',
 'bonus',
 'bps',
 'ict_index',
 'clean_sheets',
 'selected_by_percent',
 'cost',
 'nation',
 'squad',
 'age_player',
 'born',
 'mp_player',
 'starts_player',
 'min_player',
 '90s_player',
 'g-pk_player',
 'pk_player',
 'pkatt_player',
 'crdy_player',
 'crdr_player',
 'xg_player',
 'npxg_player',
 'xag_player',
 'npxg+xag_player',
 'xg_90min_player',
 'npxg_90min_player',
 'xag_90min_player',
 'npxg+xag_90min_player',
 'prgc_player',
 'prgp_player',
 'prgr_player',
 'observationcount_player',
 'num_players_team',
 'avg_age_team',
 'poss_team',
 'gls_team',
 'ast_team',
 'g+a_team',
 'g-pk_team',
 'pk_team',
 'pkatt_team',
 'crdy_team',
 'crdr_team',
 'gls_90minutes_team',
 'ast_90minutes_team',
 'g+a_90minutes_team',
 'g-pk_90minutes_team',
 'g+a-pk_team',
 'xg_team',
 'npxg_team',
 'xag_team',
 'npxg+xag_team',
 'prgc_team',
 'prgp_team',
 'xg_90min_team',
 'x